# Imports

In [1]:
import requests as rq
import pandas as pd
import os
import numpy as np
import json
from dotenv import load_dotenv
from sqlalchemy import create_engine
import copy

In [2]:
def show_cols(dict_):
    for cat in dict_.keys():
        print(f'\n\nCategory: {cat}')
        print(*[ i for i in dict_[cat].columns], sep = '\n')

## Definitions

In [3]:
base_urls = {
    "films": "https://swapi.dev/api/films/",
    "people": "https://swapi.dev/api/people/",
    "planets": "https://swapi.dev/api/planets/",
    "species": "https://swapi.dev/api/species/",
    "starships": "https://swapi.dev/api/starships/",
    "vehicles": "https://swapi.dev/api/vehicles/"
}

In [4]:
categories = list(base_urls.keys())
categories

['films', 'people', 'planets', 'species', 'starships', 'vehicles']

Each category has different fields that contain information in the form of an url. I will extract the page id from those fields for each category. 

In [5]:
fields = {
    'films' : ["characters", "planets", "starships", "vehicles", "species"],
    'people' : ["homeworld", "films", "species", "vehicles", "starships"],
    'planets' : ['residents', 'films'],
    'species' : ['people', 'films', 'homeworld'],
    'vehicles' : ['pilots', 'films'],
    'starships' : ['pilots', 'films']
    }

# Consume the API

In [6]:
def scrape_category(url):

    # skip the next url for the first page of pager
    next = url    
    items_list = []

    while next:
        # get the content of the url
        response = rq.get(next)

        # success
        if response.status_code == 200:
            content = response.json()
        elif response.status_code == 404:
            print(f'{url} not found!')
            return
        
        next = content['next']
        items = content['results']

        for item in items:
                    
            # remove created and edited fields
            try:
                del(item['created'])
                del(item['edited'])
            except:
                pass
            
            items_list.append(item)            

    return items_list

## Scrape all the categories and store in *starwars_raw.json*

In [7]:
if not os.path.exists('./data/starwars_raw.json'):
    raw_dict = {cat : scrape_category(base_urls[cat]) for cat in categories}
    
    # store into a json file
    with open('./data/starwars_raw.json', 'w') as file:
        json.dump(raw_dict, file, indent=4 )
    print('Content from Star Wars API stored in a json file!')

else:
    print('The content already exists in a json file!')
    with open('starwars_scrapped.json', 'r') as file:
        raw_dict = json.load(file)

The content already exists in a json file!


Function to process the information of an item from a category.
Ex. one character, one planet or one film.

In [8]:
def process_item(item, fields):

    # create a copy of the item dictionary
    item = copy.deepcopy(item)

    # parse the links from starships, vehicles and species
    for field in fields:
        id_values = []
                        
        if item[field]:  # if the field is not empty
            
            # parse the homeworld (just a single string value)
            if field == 'homeworld':
                # get the homeworld id
                item['homeworld'] = int(item['homeworld'].split('/')[-2])

            else:
                for link in item[field]:

                    # parse the id value in the link    
                    id_values.append(int(link.split('/')[-2]))

                # add the id values into the corresponding field key
                # convert list into tuple, as tuples are hashable
                item[field] = tuple(id_values)

                # each character belongs to only 1 species
                # that way, the species contains
        
        # if species is an empty list, it's a human character
        elif field == 'species' and not item['species']:
            item[field] = 1
                    
        # field has no values (empty list)
        else:
            item[field] = ()
                
    # remove created and edited fields
    try:
        del(item['created'])
        del(item['edited'])
    except:
        pass

    return item

# Store the processed data

In [26]:
if not os.path.exists('./data/starwars_processed_items.json'):
    # dictionary to store the processed categories
    processed_dict = {}
    
    # process each item for all the categories
    for k,v in raw_dict.items():
        items_processed = []
        for item in v:
            try:
                items_processed.append(process_item(item, fields[k]))
                processed_dict[k] = items_processed
            except:
                print(f'Error in {k}')
    
    # store the information in a json file
    with open('./data/starwars_processed_items.json', 'w') as file:
        json.dump(processed_dict, file, indent = 4)

# the file already exists, so load it
else:
    with open('./data/starwars_processed_items.json', 'r') as file:
        processed_dict= json.load(file)

        # convert lists into tuples after reading from json file
        for cat in categories:
            for item in processed_dict[cat]:
                for field in fields[cat]:
                    try:
                        item[field] = tuple(item[field])
                    # the content of the field is an integer
                    # and not a list. Cannot create a tuple from
                    # an integer using tuple(int)
                    except:
                        pass  # do not convert into a tuple, leave it as integer

    print('Processed data already existed, so the *categories_dict_processed* dictionary will be created from json file.')

Processed data already existed, so the *categories_dict_processed* dictionary will be created from json file.


# Dataframes

## Create the dataframes
Create a dictionary to store the dataframes from each category

In [27]:
dataframes = dict.fromkeys(categories)

In [28]:
for cat in categories:
    df = pd.DataFrame(processed_dict[cat])
    df['id'] = df.index + 1

    # rename columns to add '_id' to the "fields"
    rename_dict = {field : f'{field}_id' for field in fields[cat]}
    rename_dict.update({'id' : f'{cat}_id'})
    df.rename(columns = rename_dict, inplace = True)

    # reorder the columns to place id in first place
    all_columns_but_cat_id = [col for col in df.columns if col != f'{cat}_id']
    sorted_columns = [f'{cat}_id'] + all_columns_but_cat_id
    dataframes[cat] = df[sorted_columns]
#df.to_csv('./data/starwars_characters.csv', index = False)

## Rename some columns
Make some renamings to the column names

In [29]:
col_rename_dict= {
    
    'films': {
        'characters_id': 'character_id',
        'films_id': 'film_id',
        'planets_id': 'planet_id',
        'episode_id': 'episode',
        'starships_id': 'starship_id',
        'vehicles_id': 'vehicle_id',
    },
    
    'people': {
        'people_id': 'character_id',
        'films_id': 'film_id',
        'vehicles_id': 'vehicle_id',
        'starships_id': 'starship_id'
    },

    'planets': {
        'planets_id': 'planet_id',
        'films_id': 'film_id'
    },

    'species': {
        'people_id': 'character_id',
        'films_id': 'film_id'
    },

    'vehicles': {
        'pilots_id': 'pilot_id',
        'vehicles_id': 'vehicle_id',
        'films_id': 'film_id'
    },

    'starships': {
        'pilots_id': 'pilot_id',
        'starships_id': 'starship_id',
        'films_id': 'film_id'
    }
}

In [30]:
for cat in categories:
    dataframes[cat].rename(columns=col_rename_dict[cat], inplace=True)

## Clean the datasets

### Clean people

In [31]:
df = dataframes['people']
df.sample(5)

,character_id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld_id,film_id,species_id,vehicle_id,starship_id,url
18,19,Yoda,66,17,white,green,brown,896BBY,male,28,"(2, 3, 4, 5, 6)","(6,)",(),(),https://swapi.dev/api/people/20/
78,79,Tarfful,234,136,brown,brown,blue,unknown,male,14,"(6,)","(3,)",(),(),https://swapi.dev/api/people/80/
56,57,Plo Koon,188,80,none,orange,black,22BBY,male,49,"(4, 5, 6)","(26,)",(),"(48,)",https://swapi.dev/api/people/58/
6,7,Beru Whitesun lars,165,75,brown,light,blue,47BBY,female,1,"(1, 5, 6)",1,(),(),https://swapi.dev/api/people/7/
32,33,Finis Valorum,170,unknown,blond,fair,blue,91BBY,male,9,"(4,)",1,(),(),https://swapi.dev/api/people/34/


In [32]:
df.dtypes

character_id     int64
name            object
height          object
mass            object
hair_color      object
skin_color      object
eye_color       object
birth_year      object
gender          object
homeworld_id     int64
film_id         object
species_id      object
vehicle_id      object
starship_id     object
url             object
dtype: object

In [33]:
df.mass = df.mass.replace('unknown', np.nan)
df.mass = df.mass.str.replace(',', '', regex=False)
df.mass = df.mass.astype('float') 

In [34]:
df.height = df.height.replace('unknown', np.nan).astype('float')

In [35]:
df.hair_color = df.hair_color.replace('n/a', np.nan)

In [36]:
df.birth_year = df.birth_year.str.replace('BBY', ' BBY')

### Clean films

In [37]:
df = dataframes['films']
df.sample(5)

,film_id,title,episode,opening_crawl,director,producer,release_date,character_id,planet_id,starship_id,vehicle_id,species_id,url
1,2,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,"(1, 2, 3, 4, 5, 10, 13, 14, 18, 20, 21, 22, 23...","(4, 5, 6, 27)","(3, 10, 11, 12, 15, 17, 21, 22, 23)","(8, 14, 16, 18, 19, 20)","(1, 2, 3, 6, 7)",https://swapi.dev/api/films/2/
3,4,The Phantom Menace,1,Turmoil has engulfed the\r\nGalactic Republic....,George Lucas,Rick McCallum,1999-05-19,"(2, 3, 10, 11, 16, 20, 21, 32, 33, 34, 35, 36,...","(1, 8, 9)","(31, 32, 39, 40, 41)","(33, 34, 35, 36, 37, 38, 42)","(1, 2, 6, 11, 12, 13, 14, 15, 16, 17, 18, 19, ...",https://swapi.dev/api/films/4/
0,1,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","(1, 2, 3)","(2, 3, 5, 9, 10, 11, 12, 13)","(4, 6, 7, 8)","(1, 2, 3, 4, 5)",https://swapi.dev/api/films/1/
2,3,Return of the Jedi,6,Luke Skywalker has returned to\r\nhis home pla...,Richard Marquand,"Howard G. Kazanjian, George Lucas, Rick McCallum",1983-05-25,"(1, 2, 3, 4, 5, 10, 13, 14, 16, 18, 20, 21, 22...","(1, 5, 7, 8, 9)","(2, 3, 10, 11, 12, 15, 17, 22, 23, 27, 28, 29)","(8, 16, 18, 19, 24, 25, 26, 30)","(1, 2, 3, 5, 6, 8, 9, 10, 15)",https://swapi.dev/api/films/3/
4,5,Attack of the Clones,2,There is unrest in the Galactic\r\nSenate. Sev...,George Lucas,Rick McCallum,2002-05-16,"(2, 3, 6, 7, 10, 11, 20, 21, 22, 33, 35, 36, 4...","(1, 8, 9, 10, 11)","(21, 32, 39, 43, 47, 48, 49, 52, 58)","(4, 44, 45, 46, 50, 51, 53, 54, 55, 56, 57)","(1, 2, 6, 12, 13, 15, 28, 29, 30, 31, 32, 33, ...",https://swapi.dev/api/films/5/


In [38]:
df.species_id.unique()

array([(1, 2, 3, 4, 5), (1, 2, 3, 6, 7), (1, 2, 3, 5, 6, 8, 9, 10, 15),
       (1, 2, 6, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27),
       (1, 2, 6, 12, 13, 15, 28, 29, 30, 31, 32, 33, 34, 35),
       (1, 2, 3, 6, 15, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 33, 34, 35, 36, 37)],
      dtype=object)

In [39]:
df.dtypes

film_id           int64
title            object
episode           int64
opening_crawl    object
director         object
producer         object
release_date     object
character_id     object
planet_id        object
starship_id      object
vehicle_id       object
species_id       object
url              object
dtype: object

In [40]:
df.release_date = pd.to_datetime(df.release_date)

### Clean planets

In [41]:
df = categories_dataframes['planets']
df.sample(5)

NameError: name 'categories_dataframes' is not defined

In [ ]:
df.dtypes

planet_id           int64
name               object
rotation_period    object
orbital_period     object
diameter           object
climate            object
gravity            object
terrain            object
surface_water      object
population         object
residents_id       object
film_id            object
url                object
dtype: object

In [ ]:
for col in ['rotation_period', 'orbital_period', 'diameter', 'surface_water', 'population']:
    df[col] = df[col].replace('unknown', np.nan).astype('float')

In [ ]:
df.population = df.population / 1E6
df.rename(columns = {'population' : 'population_millions'}, inplace = True)

In [ ]:
df.gravity.unique()

array(['1 standard', '1.1 standard', 'N/A',
       '1.5 (surface), 1 standard (Cloud City)', '0.85 standard',
       '0.9 standard', '0.56 standard', '0.75 standard', 'unknown',
       '0.62 standard', '1', '1.56', '0.9', '0.98'], dtype=object)

In [ ]:
df.gravity = df.gravity.str.replace(' standard', '').str.replace(df.gravity[5], '1.5')
df.gravity = df.gravity.replace('unknown', np.nan)

### Clean species

In [ ]:
df = categories_dataframes['species']
df.sample(5)

,species_id,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld_id,language,character_id,film_id,url
22,23,Tholothian,mammal,sentient,unknown,dark,unknown,"blue, indigo",unknown,46,unknown,[55],"[4, 6]",https://swapi.dev/api/species/23/
12,13,Toydarian,mammal,sentient,120,"blue, green, grey",none,yellow,91,34,Toydarian,[40],"[4, 5]",https://swapi.dev/api/species/13/
13,14,Dug,mammal,sentient,100,"brown, purple, grey, red",none,"yellow, blue",unknown,35,Dugese,[41],[4],https://swapi.dev/api/species/14/
18,19,Toong,unknown,sentient,200,"grey, green, yellow",none,orange,unknown,41,Tundan,[50],"[4, 6]",https://swapi.dev/api/species/19/
25,26,Kel Dor,unknown,sentient,180,"peach, orange, red",none,"black, silver",70,49,Kel Dor,[58],"[4, 6]",https://swapi.dev/api/species/26/


In [ ]:
df.dtypes

species_id           int64
name                object
classification      object
designation         object
average_height      object
skin_colors         object
hair_colors         object
eye_colors          object
average_lifespan    object
homeworld_id        object
language            object
character_id        object
film_id             object
url                 object
dtype: object

In [ ]:
df.average_height = df.average_height.replace('unknown', np.nan).replace('n/a', np.nan).astype('float')

In [ ]:
df.average_lifespan = df.average_lifespan.replace('unknown', np.nan).replace('indefinite', 9999).astype('float')

In [ ]:
df.loc[1, 'homeworld_id'] = np.nan
df.homeworld_id = df.homeworld_id.astype('float')

### Clean vehicles

In [ ]:
df = categories_dataframes['vehicles']
df.sample(5)

,vehicle_id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,pilot_id,film_id,url
30,31,Tsmeu-6 personal wheel bike,Tsmeu-6 personal wheel bike,Z-Gomot Ternbuell Guppat Corporation,15000,3.5,330,1,1,10,none,wheeled walker,[79],[6],https://swapi.dev/api/vehicles/60/
21,22,Koro-2 Exodrive airspeeder,Koro-2 Exodrive airspeeder,Desler Gizh Outworld Mobility Corporation,unknown,6.6,800,1,1,80,unknown,airspeeder,[70],[5],https://swapi.dev/api/vehicles/45/
7,8,AT-ST,All Terrain Scout Transport,"Kuat Drive Yards, Imperial Department of Milit...",unknown,2,90,2,0,200,none,walker,[13],"[2, 3]",https://swapi.dev/api/vehicles/19/
26,27,SPHA,Self-Propelled Heavy Artillery,Rothana Heavy Engineering,unknown,140,35,25,30,500,7 days,walker,[],[5],https://swapi.dev/api/vehicles/54/
9,10,Sail barge,Modified Luxury Sail Barge,Ubrikkian Industries Custom Vehicle Division,285000,30,100,26,500,2000000,Live food tanks,sail barge,[],[3],https://swapi.dev/api/vehicles/24/


In [ ]:
df.cost_in_credits = df.cost_in_credits.replace('unknown', np.nan)
df.cost_in_credits = df.cost_in_credits.astype('float')

In [ ]:
for col in ['max_atmosphering_speed' ,'crew', 'passengers', 'cargo_capacity']:
    df[col] = df[col].replace('unknown', np.nan).replace('none', np.nan)
    df[col] = df[col].astype('float')

In [ ]:
df.length = df.length.replace('unknown', np.nan)
df.length = df.length.astype('float')

In [ ]:
df.consumables = df.consumables.replace('0', 'none')

### Clean starships

In [ ]:
df = categories_dataframes['starships']
df.sample(5)

,starship_id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilot_id,film_id,url
11,12,Imperial shuttle,Lambda-class T-4a shuttle,Sienar Fleet Systems,240000,20,850,6,20,80000,2 months,1.0,50,Armed government transport,"[1, 13, 14]","[2, 3]",https://swapi.dev/api/starships/22/
20,21,Scimitar,Star Courier,Republic Sienar Systems,55000000,26.5,1180,1,6,2500000,30 days,1.5,unknown,Space Transport,[44],[4],https://swapi.dev/api/starships/41/
27,28,Trade Federation cruiser,Providence-class carrier/destroyer,"Rendili StarDrive, Free Dac Volunteers Enginee...",125000000,1088,1050,600,48247,50000000,4 years,1.5,unknown,capital ship,"[10, 11]",[6],https://swapi.dev/api/starships/59/
13,14,Calamari Cruiser,MC80 Liberty type Star Cruiser,Mon Calamari shipyards,104000000,1200,n/a,5400,1200,unknown,2 years,1.0,60,Star Cruiser,[],[3],https://swapi.dev/api/starships/27/
4,5,Millennium Falcon,YT-1300 light freighter,Corellian Engineering Corporation,100000,34.37,1050,4,6,100000,2 months,0.5,75,Light freighter,"[13, 14, 25, 31]","[1, 2, 3]",https://swapi.dev/api/starships/10/


In [ ]:
df.loc[0, 'crew'] = 165

In [ ]:
for col in ['cost_in_credits', 'length', 'max_atmosphering_speed', 'crew', 'passengers', 'cargo_capacity', 'hyperdrive_rating', 'MGLT']:
    try:
        df[col] = df[col].replace('unknown', np.nan).replace('none', np.nan).replace('n/a', np.nan)
        df[col] = df[col].str.replace(',', '', regex = False).str.replace('km', '')
        #df[col] = df[col].astype('float')
    except Exception as e:
        print(f'Error in {col}: {e}')

In [ ]:
for col in ['cost_in_credits', 'length', 'max_atmosphering_speed', 'crew', 'passengers', 'cargo_capacity', 'hyperdrive_rating', 'MGLT']:
    try:
        df[col] = df[col].astype('float')
    except:
        print(f'error with {col}')

In [ ]:
dataframes['people']

,character_id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld_id,film_id,species_id,vehicle_id,starship_id,url
0,1,Luke Skywalker,172.0,77.0,blond,fair,blue,19 BBY,male,"(1,)","(1, 2, 3, 6)","(1,)","(14, 30)","(12, 22)",https://swapi.dev/api/people/1/
1,2,C-3PO,167.0,75.0,NaN,gold,yellow,112 BBY,n/a,"(1,)","(1, 2, 3, 4, 5, 6)","(2,)",(),(),https://swapi.dev/api/people/2/
2,3,R2-D2,96.0,32.0,NaN,"white, blue",red,33 BBY,n/a,"(8,)","(1, 2, 3, 4, 5, 6)","(2,)",(),(),https://swapi.dev/api/people/3/
3,4,Darth Vader,202.0,136.0,none,white,yellow,41.9 BBY,male,"(1,)","(1, 2, 3, 6)","(1,)",(),"(13,)",https://swapi.dev/api/people/4/
4,5,Leia Organa,150.0,49.0,brown,light,brown,19 BBY,female,"(2,)","(1, 2, 3, 6)","(1,)","(30,)",(),https://swapi.dev/api/people/5/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,Grievous,216.0,159.0,none,"brown, white","green, yellow",unknown,male,"(59,)","(6,)","(36,)","(60,)","(74,)",https://swapi.dev/api/people/79/
78,79,Tarfful,234.0,136.0,brown,brown,blue,unknown,male,"(14,)","(6,)","(3,)",(),(),https://swapi.dev/api/people/80/
79,80,Raymus Antilles,188.0,79.0,brown,light,brown,unknown,male,"(2,)","(1, 6)","(1,)",(),(),https://swapi.dev/api/people/81/
80,81,Sly Moore,178.0,48.0,none,pale,white,unknown,female,"(60,)","(5, 6)","(1,)",(),(),https://swapi.dev/api/people/82/


## Export clean datasets into csv files

In [45]:
data_path = './data'
for cat in categories:
    filename = f'{cat}_dataframe.csv'
    if os.path.exists(f'{data_path}/csv/{filename}'):
        print(f'File {filename} already exist!')
        pass
    else:
        os.makedirs(f'{data_path}/csv/', exist_ok=True)
        df = dataframes[cat]
        df.to_csv(f'{data_path}/csv/{cat}_dataframe.csv', index = False)
print(f'Dataframes of each normalized category are stored in {data_path}/csv/ as csv files!')

Dataframes of each normalized category are stored in ./data/csv/ as csv files!


# Junction tables

(many-to-many relationships in the database)

1. **films_people_junction**: Links films to the characters that appeared in them.

    - character_id: Foreign Key referencing the `people` table.
    - film_id: Foreign Key referencing the films table.

2. **films_planets_junction**: Links films to the planets that appeared in them.

    - planet_id: Foreign Key referencing the `planets` table.
    - film_id: Foreign Key referencing the `films` table.

3. **films_starships_junction**: Links films to the starships that appeared in them.

    - starship_id: Foreign Key referencing the `starships` table.
    - film_id: Foreign Key referencing the `films` table.

4. **films_vehicles_junction**: Links films to the vehicles that appeared in them.

    - vehicle_id: Foreign Key referencing the `vehicles` table.
    - film_id: Foreign Key referencing the `films` table.

5. **films_species_junction**: Links films to the species that appeared in them.

    - species_id: Foreign Key referencing the `species` table.
    - film_id: Foreign Key referencing the `films` table.

6. **people_starships_junction**: Links people (pilots) to the starships they have piloted.

    - character_id: Foreign Key referencing the `people` table.
    - starship_id: Foreign Key referencing the `starships` table.

7. **people_vehicles_junction**: Links people (pilots) to the vehicles they have piloted.

    - character_id: Foreign Key referencing the `people` table.
    - vehicle_id: Foreign Key referencing the vehicles table.

In [42]:
junction_tables = [
    'people_vehicles_junction', 
    'people_starships_junction',
    
    'films_people_junction',
    'films_planets_junction',
    'films_starships_junction',
    'films_vehicles_junction',
    'films_species_junction'
    ]

junction_tables_dict = {i:None for i in junction_tables}

## Junction tables for films:

In [43]:
data = dataframes['films'].loc[:, ['species_id', 'planet_id', 'film_id', 'vehicle_id', 'starship_id']]

In [ ]:
# junction table for people and films
junction_tables_dict['films_species_junction'] = data.explode('species_id').drop(['planet_id', 'vehicle_id', 'starship_id'], axis = 1)

# junction table for people and vehicles
junction_tables_dict['films_vehicles_junction'] = data.explode('vehicle_id').drop(['species_id', 'starship_id'], axis = 1).drop_duplicates()

# junction table for people and starships
junction_tables_dict['films_starships_junction'] = data.explode('starship_id').drop(['species_id', 'vehicle_id'], axis = 1).drop_duplicates()

## Junction tables for films:

In [45]:
data = categories_dataframes['films'].loc[:, ['character_id', 'film_id', 'vehicle_id', 'starship_id']]

# junction table for people and films
junction_tables_dict['people_films_junction'] = data.explode('character_id').drop(['vehicle_id', 'starship_id'], axis = 1).drop_duplicates()

# junction table for people and vehicles
junction_tables_dict['people_vehicles_junction'] = data.explode('vehicle_id').explode('character_id').drop(['film_id', 'starship_id'], axis = 1).drop_duplicates()

# junction table for people and starships
junction_tables_dict['people_starships_junction'] = data.explode('starship_id').explode('character_id').drop(['film_id', 'vehicle_id'], axis = 1).drop_duplicates()

In [46]:
junction_tables_dict['people_starships_junction'].head()

,character_id,starship_id
0,1,2
0,2,2
0,3,2
0,4,2
0,5,2


# Normalization
Next step is to normalize the datasets in order to create the database.

In [47]:
categories_dataframes_normalized = categories_dataframes.copy()

In [48]:
columns_to_drop = {
    'films' : ['character_id', 'planet_id', 'species_id', 'vehicle_id', 'starship_id', 'url'],
    'people' : ['film_id', 'species_id', 'vehicle_id', 'starship_id', 'url'],
    'planets' : ['residents_id', 'film_id', 'url'],
    'species' : ['character_id', 'film_id', 'url'],
    'starships' : ['pilot_id', 'film_id', 'url'],
    'vehicles' : ['pilot_id', 'film_id', 'url'],
}  

### Drop the corresponding columns in order to normalize the tables

In [49]:
for cat in categories_dataframes_normalized.keys():
    categories_dataframes_normalized[cat].drop(columns_to_drop[cat], axis='columns', inplace = True)

## Store the normalized dataframes

In [50]:
data_path = './data'
for cat in categories:
    filename = f'{cat}_dataframe.csv'
    if os.path.exists(f'{data_path}/csv_normalized/{filename}'):
        print(f'File {filename} already exist!')
        pass
    else:
        os.makedirs(f'{data_path}/csv_normalized/', exist_ok=True)
        df = categories_dataframes_normalized[cat]
        df.to_csv(f'{data_path}/csv_normalized/{cat}_dataframe_normalized.csv', index = False)
print(f'Dataframes of each normalized category are stored in {data_path}/csv_normalized/ as csv files!')

Dataframes of each normalized category are stored in ./data/csv_normalized/ as csv files!


## Example of joined people and their vehicles

In [51]:
# df2 = pd.merge(people_vehicles_junction, categories_dataframes['people'], on='people_id', how = 'inner')
# df2.rename(columns={'vehicles_id_x' : 'vehicles_id'}, inplace=True)

# df2 = pd.merge(df2, categories_dataframes['vehicles'], on='vehicles_id')
# #df2.drop(['people_id', 'vehicles_id'], axis = 1)
# df2.head()

In [52]:
# categories_dataframes['people']

# Insert data into the database

## Load database parameters from `.env` file

In [53]:
load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

## Create the db connection

In [60]:
connection_string = (
    f'mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
)

# --- 4. Create the SQLAlchemy Engine ---
try:
    engine = create_engine(connection_string)
    print("SQLAlchemy Engine created successfully. 🛠️")
except Exception as e:
    print(f"Error creating engine: {e}")

SQLAlchemy Engine created successfully. 🛠️


## Populate the data into the database

The order of tables to be filled must be:
1. planets
2. species
3. starships
4. vehicles
5. films
6. people

In [ ]:
categories_sorted = ['planets', 'species', 'vehicles', 'starships', 'films', 'people']
insert = False

### Check if the database exists and has data stored

In [70]:
def insert_category(cat, dictionary):
    df = dictionary[cat]
    try:
        df.to_sql(name=cat, con=engine, if_exists='append', index=False)
        print(f"DataFrame for category '{cat}' inserted successfully into the database. ✅\n\n")
    except Exception as e:
        print(f"\\ Error inserting DataFrame for category '{cat}': \n{e}\n\n")

In [71]:
for cat in categories_sorted:
    query = f'select * from {cat} limit 1 ;'
    # if table is empty, fill it with the corresponding data
    if pd.read_sql(query, con = engine).shape[0] == 0:
        insert_category(cat, categories_dataframes_normalized)
    else:
        print(f'{cat} table already exists in database!')

planets table already exists in database!
species table already exists in database!
vehicles table already exists in database!
starships table already exists in database!
films table already exists in database!
people table already exists in database!


In [57]:
# junction_tables_dict['people_vehicles_junction']

In [74]:
for table in junction_tables_dict.keys():
    query = f'select * from {table} limit 1 ;'
    # if table is empty, fill it with the corresponding data
    if pd.read_sql(query, con = engine).shape[0] == 0:
        insert_category(table, junction_tables_dict)
    else:
        print(f'{cat} table already exists in database!') 

ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'starwars_db.people_films_junction' doesn't exist")
[SQL: select * from people_films_junction limit 1 ;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [59]:
df = junction_tables_dict['people_vehicles_junction']
df[(df.character_id == 1) & (df.vehicle_id == 8)]

,character_id,vehicle_id
0,1,8
